In [1]:
def read_image(fname):
    with open(fname, "rb") as fp:
        lines = fp.readlines()

        for l in list(lines):
            if l[:1] == b"#":
                lines.remove(l)

        header = lines[0].decode("utf-8").strip()
        width, height = [int(i) for i in lines[1].split()]
        maxColor = int(lines[2].strip())
        arr = list()
        
        if(header == "P2"):
            for i in range(3, len(lines)):
                for line in lines[i].split():
                    arr.append(int(line))
        elif(header == "P5"):
            for i in range(3, len(lines)):
                for line in lines[i]:
                    arr.append(ord(chr(line)))
        img = list()
        k = 0
        for i in range(height):
            row = list()
            for j in range(width):
                row.append(arr[k])
                k += 1
            img.append(row)
        return img


In [2]:
def save_pgm(fname, img):
    fp = open(fname, "w")
    fp.write("P2" + "\n")
    fp.write(str(len(img[0])) + " " + str(len(img)) + "\n")
    fp.write("255\n")

    for i in range(len(img)):
        count = 0
        for j in range(len(img[i])):
            fp.write(str(img[i][j]) + " ")
            if (count > 17):
                count = 0
                fp.write("\n")
            else:
                count += 1
        fp.write("\n")
    fp.close()


In [3]:
def normalize(matrix):
    result = []
    m = min(map(min, matrix))
    for row in matrix:
        arr = []
        for val in row:
            arr.append(round(val / m))
        result.append(arr)
    return result


In [4]:
def integral(x, y, img, filter, k):
    s = 0
    for i in range(k):
        for j in range(k):
            s += img[x - k // 2 + i ][y - k // 2 + j] * filter[i][j]
    return s


In [5]:
def minMaxNormalizaiton(arr):
    mi = min(map(min, arr))
    ma = max(map(max, arr))

    for i in range(len(arr)):
        for j in range(len(arr)):
            arr[i][j] = int(((arr[i][j] - mi) / (ma - mi)) * 255)
    
    return arr

In [6]:
def convolution(img, filter):
    k = len(filter)
    arr = [[0 for i in range(len(img[j]))] for j in range(len(img))]

    for i in range(k // 2, len(img) - k // 2,):
        for j in range(k // 2, len(img[i]) - k // 2):
            arr[i][j] = integral(i, j, img, filter, k)
    
    return arr


In [7]:
def sobelXY(img_sobelX, img_sobelY):
    img_sobelXY = []
    for i in range(len(img_sobelX)):
        row = []
        for j in range(len(img_sobelX[i])):
            row.append((img_sobelX[i][j]**2+img_sobelY[i][j]**2)**0.5)
        img_sobelXY.append(row)
    return minMaxNormalizaiton(img_sobelXY)

In [28]:
gaussian_3_3_sigma_2 = [[0.093, 0.119, 0.093],
                        [0.119, 0.153, 0.119],
                        [0.093, 0.119, 0.093]]
gaussian_3_3_sigma_4 = [[0.102, 0.115, 0.102],
                        [0.115, 0.131, 0.115],
                        [0.102, 0.115, 0.102]]
gaussian_5_5_sigma_2 = [[0.012, 0.026, 0.034, 0.026, 0.012],
                        [0.026, 0.056, 0.072, 0.056, 0.026],
                        [0.034, 0.072, 0.092, 0.072, 0.034],
                        [0.026, 0.056, 0.072, 0.056, 0.026],
                        [0.012, 0.026, 0.034, 0.026, 0.012]]
gaussian_5_5_sigma_4 = [[0.023, 0.034, 0.038, 0.034, 0.023],
                        [0.034, 0.049, 0.055, 0.049, 0.034],
                        [0.038, 0.056, 0.063, 0.056, 0.038],
                        [0.034, 0.049, 0.056, 0.049, 0.034],
                        [0.023, 0.034, 0.038, 0.034, 0.023]]
gaussian_5_5_sigma_1 = [[0.003, 0.013, 0.022, 0.013, 0.003],
                        [0.013, 0.060, 0.098, 0.060, 0.013],
                        [0.022, 0.098, 0.162, 0.098, 0.022],
                        [0.013, 0.060, 0.098, 0.060, 0.013],
                        [0.003, 0.013, 0.022, 0.013, 0.003]]

gaussian_3_3_sigma_2_new = normalize(gaussian_3_3_sigma_2)
gaussian_3_3_sigma_4_new = normalize(gaussian_3_3_sigma_4)
gaussian_5_5_sigma_2_new = normalize(gaussian_5_5_sigma_2)
gaussian_5_5_sigma_4_new = normalize(gaussian_5_5_sigma_4)
gaussian_5_5_sigma_1_new = normalize(gaussian_5_5_sigma_1)

Mx = [[-1, 0, 1],
      [-2, 0, 2],
      [-1, 0, 1]]
My = [[-1, -2, -1],
      [0, 0, 0],
      [1, 2, 1]]

L1 = [[0, -1, 0],
      [-1, 4, -1],
      [0, -1, 0]]
L2 = [[-1, -1, -1],
      [-1, 8, -1],
      [-1, -1, -1]]


In [9]:
gaussian_3_3_sigma_2_new

[[1, 1, 1], [1, 2, 1], [1, 1, 1]]

In [10]:
gaussian_3_3_sigma_4_new

[[1, 1, 1], [1, 1, 1], [1, 1, 1]]

In [11]:
gaussian_5_5_sigma_2_new

[[1, 2, 3, 2, 1],
 [2, 5, 6, 5, 2],
 [3, 6, 8, 6, 3],
 [2, 5, 6, 5, 2],
 [1, 2, 3, 2, 1]]

In [12]:
gaussian_5_5_sigma_4_new

[[1, 1, 2, 1, 1],
 [1, 2, 2, 2, 1],
 [2, 2, 3, 2, 2],
 [1, 2, 2, 2, 1],
 [1, 1, 2, 1, 1]]

In [13]:
gaussian_5_5_sigma_1_new

[[1, 4, 7, 4, 1],
 [4, 20, 33, 20, 4],
 [7, 33, 54, 33, 7],
 [4, 20, 33, 20, 4],
 [1, 4, 7, 4, 1]]

In [14]:
img_lena = read_image('./images/lena.pgm')

In [15]:
img_coins = read_image('./images/coins.ascii.pgm')

In [16]:
img_lena_3_3_2 = minMaxNormalizaiton(convolution(img_lena, gaussian_3_3_sigma_2))
img_lena_3_3_4 = minMaxNormalizaiton(convolution(img_lena, gaussian_3_3_sigma_4))
img_lena_5_5_2 = minMaxNormalizaiton(convolution(img_lena, gaussian_5_5_sigma_2))
img_lena_5_5_4 = minMaxNormalizaiton(convolution(img_lena, gaussian_5_5_sigma_4))

save_pgm("./images/img_lena_3_3_2.pgm", img_lena_3_3_2)
save_pgm("./images/img_lena_3_3_4.pgm", img_lena_3_3_4)
save_pgm("./images/img_lena_5_5_2.pgm", img_lena_5_5_2)
save_pgm("./images/img_lena_5_5_4.pgm", img_lena_5_5_4)

In [17]:
img_lena_5_5_1 = minMaxNormalizaiton(convolution(img_lena, gaussian_5_5_sigma_1))
save_pgm("./images/img_lena_5_5_1.pgm", img_lena_5_5_1)

In [18]:
img_coins_3_3_2 = minMaxNormalizaiton(convolution(img_coins, gaussian_3_3_sigma_2))
img_coins_3_3_4 = minMaxNormalizaiton(convolution(img_coins, gaussian_3_3_sigma_4))
img_coins_5_5_2 = minMaxNormalizaiton(convolution(img_coins, gaussian_5_5_sigma_2))
img_coins_5_5_4 = minMaxNormalizaiton(convolution(img_coins, gaussian_5_5_sigma_4))

save_pgm("./images/img_coins_3_3_2.pgm", img_coins_3_3_2)
save_pgm("./images/img_coins_3_3_4.pgm", img_coins_3_3_4)
save_pgm("./images/img_coins_5_5_2.pgm", img_coins_5_5_2)
save_pgm("./images/img_coins_5_5_4.pgm", img_coins_5_5_4)

In [19]:
img_lena_sobelX = convolution(img_lena, Mx)
img_lena_sobelY = convolution(img_lena, My)
save_pgm("./images/img_lena_sobelX.pgm", img_lena_sobelX)
save_pgm("./images/img_lena_sobelY.pgm", img_lena_sobelY)

img_lena_sobelXY = sobelXY(img_lena_sobelX, img_lena_sobelY)
save_pgm("./images/img_lena_sobelXY.pgm", img_lena_sobelXY)

In [22]:
img_coins_sobelX = convolution(img_coins, Mx)
img_coins_sobelY = convolution(img_coins, My)
save_pgm("./images/img_coins_sobelX.pgm", img_coins_sobelX)
save_pgm("./images/img_coins_sobelY.pgm", img_coins_sobelY)

img_coins_sobelXY = sobelXY(img_coins_sobelX, img_coins_sobelY)
save_pgm("./images/img_coins_sobelXY.pgm", img_coins_sobelXY)


In [24]:
img_lena_3_3_2_sobelX = convolution(img_lena_3_3_2, Mx)
img_lena_3_3_2_sobelY = convolution(img_lena_3_3_2, My)
save_pgm("./images/img_lena_3_3_2_sobelX.pgm", img_lena_3_3_2_sobelX)
save_pgm("./images/img_lena_3_3_2_sobelY.pgm", img_lena_3_3_2_sobelY)

img_lena_3_3_2_sobelXY = sobelXY(img_lena_3_3_2_sobelX, img_lena_3_3_2_sobelY)
save_pgm("./images/img_lena_3_3_2_sobelXY.pgm", img_lena_3_3_2_sobelXY)


img_lena_3_3_4_sobelX = convolution(img_lena_3_3_4, Mx)
img_lena_3_3_4_sobelY = convolution(img_lena_3_3_4, My)
save_pgm("./images/img_lena_3_3_4_sobelX.pgm", img_lena_3_3_4_sobelX)
save_pgm("./images/img_lena_3_3_4_sobelY.pgm", img_lena_3_3_4_sobelY)

img_lena_3_3_4_sobelXY = sobelXY(img_lena_3_3_4_sobelX, img_lena_3_3_4_sobelY)
save_pgm("./images/img_lena_3_3_4_sobelXY.pgm", img_lena_3_3_4_sobelXY)


img_lena_5_5_2_sobelX = convolution(img_lena_5_5_2, Mx)
img_lena_5_5_2_sobelY = convolution(img_lena_5_5_2, My)
save_pgm("./images/img_lena_5_5_2_sobelX.pgm", img_lena_5_5_2_sobelX)
save_pgm("./images/img_lena_5_5_2_sobelY.pgm", img_lena_5_5_2_sobelY)

img_lena_5_5_2_sobelXY = sobelXY(img_lena_5_5_2_sobelX, img_lena_5_5_2_sobelY)
save_pgm("./images/img_lena_5_5_2_sobelXY.pgm", img_lena_5_5_2_sobelXY)


img_lena_5_5_4_sobelX = convolution(img_lena_5_5_4, Mx)
img_lena_5_5_4_sobelY = convolution(img_lena_5_5_4, My)
save_pgm("./images/img_lena_5_5_4_sobelX.pgm", img_lena_5_5_4_sobelX)
save_pgm("./images/img_lena_5_5_4_sobelY.pgm", img_lena_5_5_4_sobelY)

img_lena_5_5_4_sobelXY = sobelXY(img_lena_5_5_4_sobelX, img_lena_5_5_4_sobelY)
save_pgm("./images/img_lena_5_5_4_sobelXY.pgm", img_lena_5_5_4_sobelXY)

In [31]:
img_lena_3_3_2_L1 = convolution(img_lena_3_3_2, L1)
img_lena_3_3_4_L1 = convolution(img_lena_3_3_4, L1)
img_lena_5_5_2_L1 = convolution(img_lena_5_5_2, L1)
img_lena_5_5_4_L1 = convolution(img_lena_5_5_4, L1)

save_pgm("./images/img_lena_3_3_2_L1.pgm", img_lena_3_3_2_L1)
save_pgm("./images/img_lena_3_3_4_L1.pgm", img_lena_3_3_4_L1)
save_pgm("./images/img_lena_5_5_2_L1.pgm", img_lena_5_5_2_L1)
save_pgm("./images/img_lena_5_5_4_L1.pgm", img_lena_5_5_4_L1)

img_lena_3_3_2_L2 = convolution(img_lena_3_3_2, L2)
img_lena_3_3_4_L2 = convolution(img_lena_3_3_4, L2)
img_lena_5_5_2_L2 = convolution(img_lena_5_5_2, L2)
img_lena_5_5_4_L2 = convolution(img_lena_5_5_4, L2)

save_pgm("./images/img_lena_3_3_2_L2.pgm", img_lena_3_3_2_L2)
save_pgm("./images/img_lena_3_3_4_L2.pgm", img_lena_3_3_4_L2)
save_pgm("./images/img_lena_5_5_2_L2.pgm", img_lena_5_5_2_L2)
save_pgm("./images/img_lena_5_5_4_L2.pgm", img_lena_5_5_4_L2)

In [32]:
img_coins_3_3_2_sobelX = convolution(img_coins_3_3_2, Mx)
img_coins_3_3_2_sobelY = convolution(img_coins_3_3_2, My)
save_pgm("./images/img_coins_3_3_2_sobelX.pgm", img_coins_3_3_2_sobelX)
save_pgm("./images/img_coins_3_3_2_sobelY.pgm", img_coins_3_3_2_sobelY)

img_coins_3_3_2_sobelXY = sobelXY(img_coins_3_3_2_sobelX, img_coins_3_3_2_sobelY)
save_pgm("./images/img_coins_3_3_2_sobelXY.pgm", img_coins_3_3_2_sobelXY)


img_coins_3_3_4_sobelX = convolution(img_coins_3_3_4, Mx)
img_coins_3_3_4_sobelY = convolution(img_coins_3_3_4, My)
save_pgm("./images/img_coins_3_3_4_sobelX.pgm", img_coins_3_3_4_sobelX)
save_pgm("./images/img_coins_3_3_4_sobelY.pgm", img_coins_3_3_4_sobelY)

img_coins_3_3_4_sobelXY = sobelXY(img_coins_3_3_4_sobelX, img_coins_3_3_4_sobelY)
save_pgm("./images/img_coins_3_3_4_sobelXY.pgm", img_coins_3_3_4_sobelXY)


img_coins_5_5_2_sobelX = convolution(img_coins_5_5_2, Mx)
img_coins_5_5_2_sobelY = convolution(img_coins_5_5_2, My)
save_pgm("./images/img_coins_5_5_2_sobelX.pgm", img_coins_5_5_2_sobelX)
save_pgm("./images/img_coins_5_5_2_sobelY.pgm", img_coins_5_5_2_sobelY)

img_coins_5_5_2_sobelXY = sobelXY(img_coins_5_5_2_sobelX, img_coins_5_5_2_sobelY)
save_pgm("./images/img_coins_5_5_2_sobelXY.pgm", img_coins_5_5_2_sobelXY)


img_coins_5_5_4_sobelX = convolution(img_coins_5_5_4, Mx)
img_coins_5_5_4_sobelY = convolution(img_coins_5_5_4, My)
save_pgm("./images/img_coins_5_5_4_sobelX.pgm", img_coins_5_5_4_sobelX)
save_pgm("./images/img_coins_5_5_4_sobelY.pgm", img_coins_5_5_4_sobelY)

img_coins_5_5_4_sobelXY = sobelXY(img_coins_5_5_4_sobelX, img_coins_5_5_4_sobelY)
save_pgm("./images/img_coins_5_5_4_sobelXY.pgm", img_coins_5_5_4_sobelXY)

In [33]:
img_coins_3_3_2_L1 = convolution(img_coins_3_3_2, L1)
img_coins_3_3_4_L1 = convolution(img_coins_3_3_4, L1)
img_coins_5_5_2_L1 = convolution(img_coins_5_5_2, L1)
img_coins_5_5_4_L1 = convolution(img_coins_5_5_4, L1)

save_pgm("./images/img_coins_3_3_2_L1.pgm", img_coins_3_3_2_L1)
save_pgm("./images/img_coins_3_3_4_L1.pgm", img_coins_3_3_4_L1)
save_pgm("./images/img_coins_5_5_2_L1.pgm", img_coins_5_5_2_L1)
save_pgm("./images/img_coins_5_5_4_L1.pgm", img_coins_5_5_4_L1)

img_coins_3_3_2_L2 = convolution(img_coins_3_3_2, L2)
img_coins_3_3_4_L2 = convolution(img_coins_3_3_4, L2)
img_coins_5_5_2_L2 = convolution(img_coins_5_5_2, L2)
img_coins_5_5_4_L2 = convolution(img_coins_5_5_4, L2)

save_pgm("./images/img_coins_3_3_2_L2.pgm", img_coins_3_3_2_L2)
save_pgm("./images/img_coins_3_3_4_L2.pgm", img_coins_3_3_4_L2)
save_pgm("./images/img_coins_5_5_2_L2.pgm", img_coins_5_5_2_L2)
save_pgm("./images/img_coins_5_5_4_L2.pgm", img_coins_5_5_4_L2)